# Linear Regression Using PySpark

Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("LR").getOrCreate()

In [2]:
df=spark.read.csv("cruise_ship_info.csv",header=True,inferSchema=True)

In [3]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [5]:
df.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [6]:
df=df.withColumn("Pass and length",df["passengers"]*df["length"])

In [7]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|   Pass and length|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|41.223600000000005|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|41.223600000000005|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|          107.2892|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|          283.4222|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|          235.6664|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2

### I haven't used Cruise_line feature but it can be done using this

In [8]:
# from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
# indexed = indexer.fit(df).transform(df)
# indexed.head(5)

In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [10]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Pass and length']

In [11]:
assembler=VectorAssembler(
          inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
  ],
outputCol='features')

In [12]:
output=assembler.transform(df)

In [13]:
final_data=output.select("features", "crew")

In [14]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [15]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(labelCol='crew')

In [17]:
lrModel=lr.fit(train_data)

In [18]:
lrModel.coefficients

DenseVector([-0.009, 0.0124, -0.1314, 0.4219, 0.7519, 0.0028])

In [19]:
test_results = lrModel.evaluate(test_data)

In [20]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 1.3233799838689029
MSE: 1.7513345817048578
R2: 0.861780312111468


In [21]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 1.3233799838689029
MSE: 1.7513345817048578
R2: 0.861780312111468
